<a href="https://colab.research.google.com/github/yoojinlee-hub/Recommended-skills_Study_by_python/blob/main/3_1_Collaborative_Recommendations_(%ED%98%91%EB%A0%A5%EC%A0%81_%EC%B6%94%EC%B2%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = 'drive/MyDrive/practice_recommanding'
u_user_src = os.path.join(base_src,'u.user')
u_cols=['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,sep='|',names=u_cols,encoding='latin-1')
users = users.set_index('user_id')

u_item_src=os.path.join(base_src,'u.item')
i_cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventuser','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,sep='|',names=i_cols,encoding='latin-1')
movies = movies.set_index('movie_id')

u_data_src=os.path.join(base_src,'u.data')
r_cols=['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',names=r_cols,encoding='latin-1')
ratings=ratings.set_index('user_id')

base_src = 'drive/MyDrive/practice_recommanding'
u_user_src = os.path.join(base_src,'u.user')
u_cols=['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,sep='|',names=u_cols,encoding='latin-1')
users = users.set_index('user_id')

u_item_src=os.path.join(base_src,'u.item')
i_cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventuser','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,sep='|',names=i_cols,encoding='latin-1')
movies = movies.set_index('movie_id')

u_data_src=os.path.join(base_src,'u.data')
r_cols=['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',names=r_cols,encoding='latin-1')

def recom_movie(n_items):
    movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendation = recom_movies['title']
    return recommendation

#데이터 train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25,stratify=y)

#정확도를 계산하는 함수
def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

#모델별 RMSE를 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

# 정확도 계산
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating']
  # best-seller 방식
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true,y_pred)
  rmse.append(accuracy)

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id,movie_id):
  try:
    rating = train_mean[movie_id]

  except:
    rating = 3.0
  return rating

#모델별 RMSE를 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




---



In [19]:
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,stratify=y)
ratings_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')

## 코사인 유사도 검사
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity=pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

In [20]:
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.102330,0.038752,0.040294,0.278295,0.355098,0.343770,0.288855,0.077433,0.295753,...,0.265440,0.081032,0.228652,0.146992,0.169665,0.115779,0.266815,0.140182,0.129955,0.264501
2,0.102330,1.000000,0.087077,0.130782,0.070186,0.171147,0.047270,0.060653,0.135583,0.099053,...,0.167472,0.211298,0.273995,0.346645,0.186760,0.142790,0.130549,0.122683,0.137614,0.058740
3,0.038752,0.087077,1.000000,0.167258,0.000000,0.056561,0.061125,0.063025,0.023120,0.026828,...,0.042618,0.000000,0.138978,0.018472,0.067573,0.019783,0.114236,0.084988,0.096920,0.015259
4,0.040294,0.130782,0.167258,1.000000,0.013382,0.068991,0.070367,0.190642,0.060099,0.054718,...,0.024619,0.050314,0.050845,0.123473,0.109676,0.041141,0.221065,0.201987,0.140426,0.052888
5,0.278295,0.070186,0.000000,0.013382,1.000000,0.208868,0.303918,0.190501,0.075534,0.170621,...,0.266473,0.028392,0.067267,0.028151,0.116042,0.029547,0.220038,0.131141,0.099157,0.218405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.115779,0.142790,0.019783,0.041141,0.029547,0.086686,0.067658,0.126839,0.000000,0.065483,...,0.045426,0.344571,0.210807,0.179148,0.345394,1.000000,0.052686,0.161265,0.041036,0.110725
940,0.266815,0.130549,0.114236,0.221065,0.220038,0.281948,0.226951,0.218142,0.053019,0.199074,...,0.203173,0.090205,0.111187,0.137429,0.169713,0.052686,1.000000,0.127537,0.202163,0.145241
941,0.140182,0.122683,0.084988,0.201987,0.131141,0.139156,0.049431,0.112438,0.052351,0.099299,...,0.061653,0.216395,0.222906,0.155355,0.289219,0.161265,0.127537,1.000000,0.071953,0.067664


In [22]:
#### 주어진 영화의 가중평균 rating을 계산하는 함수
def CF_simple(user_id,movie_id):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_ratings = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating
